In [1]:
import numpy as np
from sklearn import datasets

#GD를 활용한 LogisticRegression
class LogisticRegression:
    def __init__(self, learning_rate=0.01, threshold=0.01, max_iterations=100000, fit_intercept=True, verbose=False):
        self._learning_rate = learning_rate  # 학습 계수
        self._max_iterations = max_iterations  # 반복 횟수
        self._threshold = threshold  # 학습 중단 계수
        self._fit_intercept = fit_intercept  # 절편 사용 여부를 결정
        self._verbose = verbose  # 중간 진행사항 출력 여부

    # theta(W) 계수들 return
    def get_coeff(self):
        return self._W

    # 절편 추가
    def add_intercept(self, x_data): # intercept. feature 외 변수.
        intercept = np.ones((x_data.shape[0], 1))
        return np.concatenate((intercept, x_data), axis=1)

    # 시그모이드 함수(로지스틱 함수)
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def cost(self, h, y):##어떤뜻인지 파악하기(cost function에 대한 이해필요(GD,SGD 찾아보기)) * Log Likelihood의 음수는 목적함수. lo
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    
    
    def fit(self, x_data, y_data):
        num_examples, num_features = np.shape(x_data)

        if self._fit_intercept:
            x_data = self.add_intercept(x_data)

        
        self._W = np.zeros(x_data.shape[1])

        for i in range(self._max_iterations):
            z = np.dot(x_data, self._W) # 독립변수(x_data)와 계수(coeff, _W)를 내적한다
            hypothesis = self.sigmoid(z) # 내적한 결과를 시그모이드 함수(로지스틱 함수)에 넣어 음의 무한대와 양의 무한대 사이의 살수에서 0과 1사이의 실수로 변환하여 y_hat(hypothesis)를 도출한다.

            #실제값과 예측값의 차이
            diff = hypothesis - y_data
            cost = self.cost(hypothesis, y_data)

            ##어떤 과정인지 설명하기 * 우도 함수에 로그를 씌운 함수의 음수는 목적함수와 같다. 
            gradient = np.dot(x_data.transpose(), diff) / num_examples
            self._W -= self._learning_rate * gradient

            if cost < self._threshold:
                return False
           
            if (self._verbose == True and i % 100 == 0):
                print('cost :', cost)

    def predict_prob(self, x_data):
        if self._fit_intercept:
            x_data = self.add_intercept(x_data)

        return self.sigmoid(np.dot(x_data, self._W))

    def predict(self, x_data):
        
        return self.predict_prob(x_data).round()##왜 라운드 함수를 쓰는지 * 0과 1사이의 실수를 0 혹은 1로 분류하기 위해서다.

In [2]:
!ls

Social_Network_Ads.csv         Test logistic regression.ipynb


In [3]:
data_raw = pd.read_csv('Social_Network_Ads.csv')
data_raw.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [4]:
data_raw.shape

(400, 5)

In [5]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
User ID            400 non-null int64
Gender             400 non-null object
Age                400 non-null int64
EstimatedSalary    400 non-null int64
Purchased          400 non-null int64
dtypes: int64(4), object(1)
memory usage: 15.7+ KB


### preprocessing

In [6]:
data_raw['Male'] = data_raw['Gender'].apply(lambda x : 1 if x == 'Male' else 0)
data_raw['Female'] = data_raw['Gender'].apply(lambda x : 1 if x == 'Female' else 0)

In [7]:
data_raw.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased,Male,Female
395,15691863,Female,46,41000,1,0,1
396,15706071,Male,51,23000,1,1,0
397,15654296,Female,50,20000,1,0,1
398,15755018,Male,36,33000,0,1,0
399,15594041,Female,49,36000,1,0,1


In [9]:
data_revise = data_raw.drop(['User ID', 'Gender'], axis=1)
X_data = data_revise.drop('Purchased', axis=1)
y_data = data_revise['Purchased']
data_revise.tail()

,Age,EstimatedSalary,Purchased,Male,Female
395,46,41000,1,0,1
396,51,23000,1,1,0
397,50,20000,1,0,1
398,36,33000,0,1,0
399,49,36000,1,0,1


In [10]:
data_revise.Purchased.value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [17]:
LR_1 = LogisticRegression(learning_rate=0.00000001, threshold=0.01, max_iterations=10000, verbose=True)
LR_1.fit(X_data, y_data)
y_pred = LR_1.predict(X_data)

cost : 0.6931471805599452
cost : 0.6884784122636424
cost : 5.808220684400094
cost : 5.757929640613115
cost : 0.689362731758335
cost : 5.581297293844038
cost : 0.6887517583586744
cost : 5.082118306508232
cost : 0.6979899907711521
cost : 7.150969081848641
cost : 8.906996354947138
cost : 7.893910656189998
cost : 0.6879346482051831
cost : 8.992807832632112
cost : 7.845312072986257
cost : 7.636441991396196
cost : 0.689711451665371
cost : 7.727826442454078
cost : 1.9249368642343985
cost : 0.6901370844712786
cost : 5.282025084776339
cost : 2.5964186620793277
cost : 6.300923734634966
cost : 0.6894047891133522
cost : 9.166186790287815
cost : 0.7423821902802388
cost : 2.6009102195390965
cost : 1.0739486273644514
cost : 7.951064041972142
cost : 8.9815946248324
cost : 7.686431263104868
cost : 4.490934078231643
cost : 9.049440672484064
cost : 9.568412790220888
cost : 2.693362683978842
cost : 3.668339982556806
cost : 2.0209148719467636
cost : 8.995186528109553
cost : 8.821192776527745
cost : 4.86937

KeyboardInterrupt: 

In [53]:
LR_1.get_coeff()

array([-9.57514598e-06, -4.45062196e-05, -2.67296399e-06, -5.77730430e-06,
       -3.79784167e-06])

In [54]:
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.